# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

In [1]:
# %cd /mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT
# !mkdir -p models
# %cd ./models
# !mkdir -p sentiments
# %cd ./sentiments
# !wget https://prosus-public.s3-eu-west-1.amazonaws.com/finbert/finbert-sentiment/pytorch_model.bin

In [2]:
# import pandas as pd
# from sklearn.model_selection import train_test_split

# #index=False
# column_names=['label','text']

# data = pd.read_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/learning_data.csv", sep=',', engine='python')
# train, test = train_test_split(data, test_size=0.2, random_state=0)
# train, valid = train_test_split(train, test_size=0.1, random_state=0)
# train.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/train.csv",sep='\t', header=column_names)
# test.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/test.csv",sep='\t', header=column_names)
# valid.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/validation.csv",sep='\t', header=column_names)

In [3]:
# import csv
# import pandas as pd
# from sklearn.model_selection import train_test_split

# #data = pd.DataFrame({'text':[],'label':[]})
# data = pd.DataFrame(columns=['text','label'])


# with open("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/learning_data.csv") as f:
#     reader = csv.reader(f)
#     rows = list(reader)
#     for i, item in enumerate(rows):
#         df = pd.DataFrame({'text':[item[1]],'label':[item[0]]})
#         data = data.append(df, ignore_index=True)

# column_names=['text','label']       
# train, test = train_test_split(data, test_size=0.2, random_state=0)
# train, valid = train_test_split(train, test_size=0.1, random_state=0)
# train.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/train.csv",sep='\t', header=column_names)
# test.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/test.csv",sep='\t', header=column_names)
# valid.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/validation.csv",sep='\t', header=column_names)

In [4]:
# import csv
# import pandas as pd
# from sklearn.model_selection import train_test_split

# data = pd.DataFrame(columns=['text','label'])

# with open("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/learning_data.csv") as f:
#     reader = csv.reader(f)
#     rows = list(reader)
#     g46=0
#     g62=0
#     g510=0
    
#     for i, item in enumerate(rows):
        
#         word_list=item[1].split()
#         if len(word_list)>46:
#             g46+=1
#             #item[1]=' '.join(word_list[0:46])
#         if len(word_list)>62:
#             g62+=1
#             item[1]=' '.join(word_list[0:62])
#         if len(word_list)>510:
#             g510+=1
#             #item[1]=' '.join(word_list[0:510])
            
#         df = pd.DataFrame({'text':[item[1]],'label':[item[0]]})
#         data = data.append(df, ignore_index=True)

# print('g46:',g46)
# print('g62:',g62)
# print('g510:', g510)

# column_names=['text','label']       
# train, test = train_test_split(data, test_size=0.2, random_state=0)
# train, valid = train_test_split(train, test_size=0.1, random_state=0)
# train.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/train.csv",sep='\t', header=column_names)
# test.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/test.csv",sep='\t', header=column_names)
# valid.to_csv("/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/data/sentiment_data/validation.csv",sep='\t', header=column_names)

## Modules 

In [5]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

In [6]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [7]:
# lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
# cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
# cl_data_path = project_dir/'data'/'sentiment_data'

print(project_dir)
lm_path = project_dir/"models"/"language"
cl_path = project_dir/"models"/"sentiment"
cl_data_path = project_dir/"data"/"sentiment_data"

/mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT


###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [8]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 64,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

Some weights of the model checkpoint at /mnt/8a7936f5-422f-42ea-980d-d68370db7589/AiProj2/finbert/finBERT/models/language were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequence

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [9]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [10]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

05/07/2021 12:55:40 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


## Fine-tune the model

In [11]:
# Get the training examples
train_data = finbert.get_data('train')

In [12]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [13]:
# # This is for fine-tuning a subset of the model.

# freeze = 6

# for param in model.bert.embeddings.parameters():
#     param.requires_grad = False
    
# for i in range(freeze):
#     for param in model.bert.encoder.layer[i].parameters():
#         param.requires_grad = False

### Training

In [14]:
trained_model = finbert.train(train_examples = train_data, model = model)

05/07/2021 12:55:44 - INFO - finbert.utils -   *** Example ***
05/07/2021 12:55:44 - INFO - finbert.utils -   guid: train-1
05/07/2021 12:55:44 - INFO - finbert.utils -   tokens: [CLS] p & g rest ##ru ##cture ##s operations , creates six business units ( reuters largest geographic markets , its biggest organizational change in the last 20 years that will be effective from july next year . the maker of tide deter ##gent and oral - b tooth ##pas ##te said the new divisions - fabric and home care , baby and [SEP]
05/07/2021 12:55:44 - INFO - finbert.utils -   input_ids: 101 1052 1004 1043 2717 6820 14890 2015 3136 1010 9005 2416 2449 3197 1006 26665 2922 9183 6089 1010 2049 5221 13296 2689 1999 1996 2197 2322 2086 2008 2097 2022 4621 2013 2251 2279 2095 1012 1996 9338 1997 10401 28283 11461 1998 8700 1011 1038 11868 19707 2618 2056 1996 2047 5908 1011 8313 1998 2188 2729 1010 3336 1998 102
05/07/2021 12:55:44 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

Iteration:   0%|          | 0/643 [00:00<?, ?it/s]

05/07/2021 13:01:46 - INFO - finbert.utils -   *** Example ***
05/07/2021 13:01:46 - INFO - finbert.utils -   guid: validation-1
05/07/2021 13:01:46 - INFO - finbert.utils -   tokens: [CLS] moe ##lis hires former la ##zard banker rent ##on for oil and gas team : banker mark rent ##on to work in its energy team , according to a memo seen by reuters , the contents of which were confirmed by a spoke ##sw ##oman . based in new york as of this week , rent ##on joined moe ##lis from fellow [SEP]
05/07/2021 13:01:46 - INFO - finbert.utils -   input_ids: 101 22078 6856 28208 2280 2474 26154 13448 9278 2239 2005 3514 1998 3806 2136 1024 13448 2928 9278 2239 2000 2147 1999 2049 2943 2136 1010 2429 2000 1037 24443 2464 2011 26665 1010 1996 8417 1997 2029 2020 4484 2011 1037 3764 26760 20778 1012 2241 1999 2047 2259 2004 1997 2023 2733 1010 9278 2239 2587 22078 6856 2013 3507 102
05/07/2021 13:01:46 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

Validating:   0%|          | 0/72 [00:00<?, ?it/s]

Validation losses: [1.035332317981455]
No best model found


...

Validation losses: [1.035332317981455, 0.9695671109689606, 0.965797489715947]


Epoch:  75%|███████▌  | 3/4 [32:23<11:47, 707.25s/it]

Iteration:   0%|          | 0/643 [00:00<?, ?it/s]

Epoch:  75%|███████▌  | 3/4 [37:31<12:30, 750.54s/it]


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.95 GiB total capacity; 3.42 GiB already allocated; 10.69 MiB free; 96.57 MiB cached)

## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [ ]:
test_data = finbert.get_data('test')

In [ ]:
results = finbert.evaluate(examples=test_data, model=trained_model)

### Prepare the classification report

In [ ]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [ ]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [ ]:
report(results,cols=['labels','prediction','predictions'])

### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [ ]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [ ]:
#cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
result = predict(text,model)

In [ ]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

In [ ]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Here is another example

In [ ]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [ ]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

In [ ]:
result2

In [ ]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))